In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Plotly Libraris
import plotly.express as px
import plotly.graph_objects as go
#import plotly.figure_factory as ff
#from plotly.colors import n_colors

from plotly.subplots import make_subplots
# Minmax scaler
from sklearn.preprocessing import MinMaxScaler

#itertools
import itertools

#dataframe display settings
pd.set_option('display.max_columns', 5000000)
pd.set_option('display.max_rows', 50000000)

#to suppress un-necessary warnings
import warnings  
warnings.filterwarnings('ignore')


import xgboost as xgb

#Importing SKlearn models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression  
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

#Package to flatten python lists
from pandas.core.common import flatten

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Data

* All datasets are sourced from [COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19) which is updated daily by them.
* Click [here](https://coronavirus.jhu.edu/map.html) to visit dashboard created by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University.

In [ ]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/08-22-2020.csv')
us_medical_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/08-22-2020.csv')

# Preprocessing

In [ ]:
confirmed_group_df = confirmed_df.groupby(by='Country/Region',as_index=False).sum()
deaths_group_df = deaths_df.groupby(by='Country/Region',as_index=False).sum()
recoveries_group_df = recoveries_df.groupby(by='Country/Region',as_index=False).sum()

active_group_df = pd.DataFrame(columns=[confirmed_group_df.columns])
active_group_df = deaths_group_df.copy()
for i in range(confirmed_group_df.shape[0]):
    for j in range(3, confirmed_group_df.shape[1]):
        active_group_df.iloc[i,j] = confirmed_group_df.iloc[i,j]-(recoveries_group_df.iloc[i,j]+deaths_group_df.iloc[i,j])

In [ ]:
deaths_group_df_sorted_df_30 = deaths_group_df.sort_values(by=deaths_group_df.columns[-2], ascending=False).iloc[0:30]
del deaths_group_df_sorted_df_30['Lat']
del deaths_group_df_sorted_df_30['Long']
del confirmed_group_df['Lat']
del confirmed_group_df['Long']


In [ ]:
countries = list(deaths_group_df_sorted_df_30['Country/Region'])

# filtering out to only the top 30 countries with death
sorted_confirmed_df = confirmed_group_df.loc[confirmed_group_df['Country/Region'].isin(countries)]

new_df = pd.DataFrame(columns=['Country','Continent','Date','Deaths on Date','Confirmed Cases on date','Proportion of Males','Cumulative complete Vaccination to date','Average Age'])
n_df_columns = deaths_group_df_sorted_df_30.columns

In [ ]:
for dates_i in range(1,deaths_group_df_sorted_df_30.shape[1]):
    for country_i in range(deaths_group_df_sorted_df_30.shape[0]):
        new_row = {'Date':n_df_columns[dates_i], 'Country' : deaths_group_df_sorted_df_30.iloc[country_i,0], 'Deaths on Date' : deaths_group_df_sorted_df_30.iloc[country_i,dates_i]
                  ,'Confirmed Cases on date' : sorted_confirmed_df.iloc[country_i,dates_i] }
        new_df = new_df.append(new_row,ignore_index=True)

In [ ]:
new_df = pd.read_csv('../input/task1countriesd1/draft1_task1.csv')
del new_df['Unnamed: 0']

### Preprocessing Continents

In [ ]:
cc_map = {'US': 'North America',
 'Brazil': 'South America',
 'India': 'Asia',
 'Mexico': 'North America',
 'Russia': 'Europe',
 'Peru': 'South America',
 'Indonesia': 'Asia',
 'United Kingdom': 'Europe',
 'Italy': 'Europe',
 'Colombia': 'South America',
 'Iran': 'Asia',
 'France': 'Europe',
 'Argentina': 'South America',
 'Germany': 'Europe',
 'South Africa': 'Africa',
 'Spain': 'Europe',
 'Poland': 'Europe',
 'Turkey': 'Asia',
 'Ukraine': 'Europe',
 'Romania': 'Europe',
 'Philippines': 'Asia',
 'Chile': 'South America',
 'Ecuador': 'South America',
 'Czechia': 'Europe',
 'Hungary': 'Europe',
 'Canada': 'North America',
 'Pakistan': 'Asia',
 'Malaysia': 'Asia',
 'Bangladesh': 'Asia',
 'Belgium': 'Europe'}

# applying mapping of continents
new_df['Continent'] = new_df['Country'].map(cc_map)

In [ ]:
# Label encoding for continents and countries

# One Hot encoding
encoded_df = pd.get_dummies(new_df, columns=["Continent"]).head()

from sklearn.preprocessing import LabelEncoder

# using lable_encoder
labelencoder = LabelEncoder()
label_encoded_df = new_df.iloc[:,:]
label_encoded_df['Continent'] = labelencoder.fit_transform(label_encoded_df['Continent'])



#### using lable_encoder


In [ ]:
labelencoder = LabelEncoder()
label_encoded_df = new_df.iloc[:,:]
label_encoded_df['Continent'] = labelencoder.fit_transform(label_encoded_df['Continent'])

In [ ]:
label_encoded_df = label_encoded_df[label_encoded_df['Confirmed Cases on date'].notna()]
out = label_encoded_df.iloc[:,:5].reset_index()
del out['index']

### Continent data 
Update with GDP


In [ ]:
import json
import pandas as pd
gdp_dict = json.load(open('../input/task1countriesd1/gdp (2).json'))
cont_df = pd.read_csv('../input/task1countriesd1/continent_update.csv')
del cont_df['Unnamed: 0']


In [ ]:
countries = list(cont_df['Country'])
gdp_country_map = {}
for i in gdp_dict:
    gdp = i['2021 GDP']
    country = i["Country"]
    try:
        gdp_country_map[country] = float(gdp)
    except:
        continue


In [ ]:
counter = 0
for i in gdp_country_map.keys():
    if i in countries:
        counter +=1
    else:
        print(i)
counter

In [ ]:
cont_df['GDP Per Capita'] = cont_df['Country'].map(gdp_country_map)

In [ ]:
cont_df = cont_df.dropna()

In [ ]:
len(set(cont_df['Country']))

In [ ]:
cont_df.head()

In [ ]:
cont_df.to_csv('gdp_update.csv')

In [ ]:
import numpy as np
np.where(pd.isnull(cont_df))

# Modelling with Linear Regression

### Loading Data

In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv("datetime_update.csv")

In [5]:
df.sort_values(by=['Deaths on Date'],ascending=False)

In [9]:
df['Date'] = df['Date'].apply(pd.to_datetime)

In [11]:
# calculate deaths per day

## Linear Regression Models


Simple Linear Regression

`y = b0 + b1 * x`

where b0 and b1 are the coefficients we must estimate from the training data.
```
B1 = sum((x(i) - mean(x)) * (y(i) - mean(y))) / sum( (x(i) - mean(x))^2 )
B0 = mean(y) - B1 * mean(x)
```

Calculating Mean and Variance

Mean: 

`mean(x) = sum(x) / count(x)`

Variance:

`variance = sum( (x - mean(x))^2 )`

# Data Viz


## Deaths over time (Macro)

In [ ]:
base_stats = pd.DataFrame(columns=['Dates','Confirmed','Deaths','Recovered','Active'])
base_stats['Dates'] = confirmed_df.columns[4:]

base_stats['Confirmed'] = base_stats['Dates'].apply(lambda x: confirmed_df[x].sum())
base_stats['Deaths'] = base_stats['Dates'].apply(lambda x: deaths_df[x].sum())
base_stats['Recovered'] = base_stats['Dates'].apply(lambda x: recoveries_df[x].sum())
base_stats.reset_index(drop=False, inplace=True)
base_stats['Active'] = base_stats['index'].apply(lambda x: (base_stats['Confirmed'][x]-(base_stats['Deaths'][x]+base_stats['Recovered'][x])))
base_stats.head()

In [ ]:


base_stats_fig = go.Figure() # create canvas

for column in base_stats.columns.to_list()[2:6]:
    color_dict = {
      "Deaths": "#073b4c"
        }
    base_stats_fig.add_trace(
        go.Scatter(
            x = base_stats['Dates'],
            y = base_stats[column],
            name = column,
            line = dict(color=color_dict[column]),
            hovertemplate ='<br><b>Date</b>: %{x}'+'<br><i>Count</i>:'+'%{y}',
        )
    )
    
for column in base_stats.columns.to_list()[2:6]:
    color_dict = {
      "Deaths": "#0C6583"
        }
    base_stats_fig.add_trace(
        go.Scatter(
            x = base_stats['Dates'],
            y = base_stats['index'].apply(lambda x: (base_stats[column][x-7:x].sum())/7 if x>7 else (base_stats[column][0:x].sum())/7),
            name = column+" 7-day Moving Avg.",
            line = dict(dash="dash", color=color_dict[column]), showlegend=False,
            hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>7-day moving avg.</i>: %{y}'
        )
    )
    
base_stats_fig.update_layout(
    updatemenus=[
        dict(
        buttons=list(
            [
             dict(label = 'Deaths',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, True, False, False]},
                          {'title': 'Deaths',
                           'showlegend':True}]),
            ]),
             type = "dropdown",
             direction="down",
#             pad={"r": 10, "t": 40},
             showactive=True,
             x=0,
             xanchor="left",
             y=1.25,
             yanchor="top"
        ),
        dict(
        buttons=list(
            [dict(label = 'Linear Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'linear'}},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Log Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'log'}},
                          {'title': 'Confirmed',
                           'showlegend':True}]),
            ]),
             type = "dropdown",
             direction="down",
#             pad={"r": 10, "t": 10},
             showactive=True,
             x=0,
             xanchor="left",
             y=1.36,
             yanchor="top"
        )
    ])


base_stats_fig.update_xaxes(showticklabels=False)
base_stats_fig.update_layout(
    #height=600, width=600, 
    title_text="Basic Statistics for Covid19", title_x=0.5, title_font_size=20,
                            legend=dict(orientation='h',yanchor='top',y=1.15,xanchor='right',x=1), paper_bgcolor="mintcream",
                            xaxis_title="Date", yaxis_title="# of Cases")
base_stats_fig.show()

## Country Wise Stats

In [ ]:
country_data = go.Figure()
country_data.add_trace(go.Table(
    header=dict(values=['Country','Confirmed','Active','Recovered','Deaths','Daily Increase','Mortality Rate'],
                fill = dict(color='#A5B3F3'),
                line_color='darkslategray',
                align = ['left'] * 5),
    cells=dict(values=[confirmed_group_df.sort_values(by=confirmed_group_df.columns[-1], ascending=False)['Country/Region'], 
                      confirmed_group_df.sort_values(by=confirmed_group_df.columns[-1], ascending=False)['Country/Region'].apply(lambda x: confirmed_group_df[confirmed_group_df['Country/Region']==x][confirmed_group_df.columns[4:]].values.tolist()[0][-1]),
                      confirmed_group_df.sort_values(by=confirmed_group_df.columns[-1], ascending=False)['Country/Region'].apply(lambda x: active_group_df[active_group_df['Country/Region']==x][active_group_df.columns[4:]].values.tolist()[0][-1]),
                      confirmed_group_df.sort_values(by=confirmed_group_df.columns[-1], ascending=False)['Country/Region'].apply(lambda x: recoveries_group_df[recoveries_group_df['Country/Region']==x][recoveries_group_df.columns[4:]].values.tolist()[0][-1]),
                      confirmed_group_df.sort_values(by=confirmed_group_df.columns[-1], ascending=False)['Country/Region'].apply(lambda x: deaths_group_df[deaths_group_df['Country/Region']==x][deaths_group_df.columns[4:]].values.tolist()[0][-1]),
                      confirmed_group_df.sort_values(by=confirmed_group_df.columns[-1], ascending=False)['Country/Region'].apply(lambda x: confirmed_group_df[confirmed_group_df['Country/Region']==x][confirmed_group_df.columns[4:]].values.tolist()[0][-1]-confirmed_group_df[confirmed_group_df['Country/Region']==x][confirmed_group_df.columns[4:]].values.tolist()[0][-2]),
                      confirmed_group_df.sort_values(by=confirmed_group_df.columns[-1], ascending=False)['Country/Region'].apply(lambda x: (deaths_group_df[deaths_group_df['Country/Region']==x][deaths_group_df.columns[4:]].values.tolist()[0][-1]/confirmed_group_df[confirmed_group_df['Country/Region']==x][confirmed_group_df.columns[4:]].values.tolist()[0][-1])*100).round(decimals=3)
                      ],
               fill = dict(color='#F0FCFD'),
               line_color='darkslategray',
               align = ['left'] * 5)))

country_data.update_layout(
    #height=600, width=1100, 
    title_text="Country wise stats",
                                     title_x=0.5, title_font_size=20,
                                     paper_bgcolor="mintcream")
country_data.show()


In [ ]:
treemap_fig = go.Figure()
df_dict={
  "Confirmed": [confirmed_group_df,True],
  "Active": [active_group_df,False],
  "Recovered": [recoveries_group_df,False],
  "Deaths": [deaths_group_df,False],
  "Daily_inc": [None,False]
}
for column in ['Confirmed','Active','Recovered','Deaths']:

    treemap_fig.add_trace(go.Treemap(labels = confirmed_group_df['Country/Region'], name="Treemap",
                                     parents = ['']*confirmed_group_df.shape[0],
                                     values = df_dict[column][0][confirmed_group_df.columns[-1]],
                                     branchvalues="total",
                                     textinfo = "percent root+label+value+text", outsidetextfont = {"size": 30, "color": "darkblue"},
                                     marker = {"line": {"width": 2}}, pathbar = {"visible": False}, visible = df_dict[column][1], 
                                     hovertemplate='<b>%{label} </b> <br> Count: %{value}<br>'
                                     )) 
    
treemap_fig.add_trace(go.Treemap(labels = confirmed_group_df.sort_values(by=confirmed_group_df.columns[-1], ascending=False)['Country/Region'], name="Treemap",
                                 parents = ['']*confirmed_group_df.shape[0],
                                 values = confirmed_group_df.sort_values(by=confirmed_group_df.columns[-1], ascending=False)['Country/Region'].apply(lambda x: confirmed_group_df[confirmed_group_df['Country/Region']==x][confirmed_group_df.columns[4:]].values.tolist()[0][-1]-confirmed_group_df[confirmed_group_df['Country/Region']==x][confirmed_group_df.columns[4:]].values.tolist()[0][-2]),
                                 branchvalues="total",
                                 textinfo = "percent root+label+value+text", outsidetextfont = {"size": 30, "color": "darkblue"},
                                 marker = {"line": {"width": 2}}, pathbar = {"visible": False}, visible = df_dict['Daily_inc'][1], 
                                 hovertemplate='<b>%{label} </b> <br> Count: %{value}<br>'
                                 )) 

treemap_fig.update_layout(
    updatemenus=[
        dict(
        buttons=list([
             dict(label = 'Confirmed',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, False]},
                          {'title': 'Confirmed',
                           'showlegend':True}]),
             dict(label = 'Active',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False]},
                          {'title': 'Active',
                           'showlegend':True}]),
             dict(label = 'Recovered',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False]},
                          {'title': 'Recovered',
                           'showlegend':True}]),
             dict(label = 'Deaths',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False]},
                          {'title': 'Deaths',
                           'showlegend':True}]),
            dict(label = 'Daily Increase',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True]},
                          {'title': 'Daily Increase',
                           'showlegend':True}]),
            ]),
             type = "buttons",
             direction="down",
#             pad={"r": 10, "t": 40},
             showactive=True,
             x=1.01,
             xanchor="left",
             y=0.8,
             yanchor="top"
        )
    ])

treemap_fig.update_layout(
    #height=600, width=1100, 
    title_text="Treemap of Countries <br> The Treemap shows the number of Cases in Different coutries <br> and their percent of total cases worldwide",
                          title_x=0.5, title_font_size=15,
                          legend=dict(orientation='h',yanchor='top',y=1.12,xanchor='right',x=1), paper_bgcolor="mintcream")
treemap_fig.show()

# Linear Regression

In [ ]:
# Linear Regression using Scikit Learn

import numpy as np
from sklearn.linear_model import LinearRegression

Deaths as the only parameter

In [ ]:
death_pred_df = deaths_group_df.copy()
death_pred_df["Cumulative Deaths"] = deaths_group_df.loc[:,'1/22/20':].sum(axis= 1)
from datetime import datetime
start_date = datetime.strptime('1/22/20','%m/%d/%y')
end_date = datetime.strptime(deaths_df.columns[-1],'%m/%d/%y')
delta_t = end_date - start_date

death_pred_df["Time Period (days)"] = delta_t.days

death_pred_df_sorted_30 = death_pred_df.sort_values(by=death_pred_df.columns[-2], ascending=False).iloc[0:30].reset_index(drop=True)

In [ ]:
death_pred_df_sorted_30.head()

Sample with USA

In [ ]:
usa_death_pred = death_pred_df_sorted_30.iloc[0]
# Convert days to days from start
usa_death_pred['days_from_start'] = (df.index - df.index[0]).days; df


In [ ]:
usa_death_pred = death_pred_df_sorted_30.iloc[0]
usa_death_pred.to_csv('usa_p.csv')

In [ ]:
usa_death_pred

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8063f459-52be-4c78-9eaa-2f01d373f9b4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>